#### Problem 3 Code & Results
Want to solve the following SOC problem
$$
\begin{array}{rl}
\mbox{max} & z \\
\mbox{s.t} & {\boldsymbol \mu}\{k\}^\top\mathbf{x} - \lambda\lVert\mathbf{V}\{k\}\mathbf{x}\rVert_{2} \geq z ~, \\
           & \sum_{i=1}^{n}(x_i^+ - x_i^-) = 1 ~ , \\ 
           & \sum_{i=1}^{n}x_i^- \leq 4 ~ ,\\
           & x_k^{+}, x_k^{-} \geq 0~, \ \ \forall k = 1, \ldots , n
 \end{array}
 $$

In [1]:
import numpy as np
import scipy.io as sio
import cvxpy as cvx
import gurobi as grb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mat_contents = sio.loadmat('multiscenario.mat')

In [3]:
_mu = mat_contents['mu']
_V = mat_contents['V']
_lambda = mat_contents['lambda']

# convert into numpy arrays 
m = _mu.shape[0] # number of scenarios
d = _mu[0][0].shape[0] # dimension of the problem

mu = np.empty((m,d))
V = np.empty((m,d,d))
for i in range(m):
    mu[i,:] = np.ravel(_mu[i][0])
    V[i,:,:] = _V[i][0]

In [4]:
#r = np.max(mu.ravel())/2
#B = 10

x = cvx.Variable(d)
beta = cvx.Variable(1)
x_pos = cvx.Variable(d)
x_neg = cvx.Variable(d)
x = x_pos - x_neg

objective = cvx.Maximize(beta)
constraints = []

# volatility constriants
for k in range(m):
    constraints += [mu[k,:].T*x - _lambda*cvx.norm(V[k,:,:]*x) >= beta];

# mean constraints
#for k in range(m):
#    constraints += [mu[k,:].T*x >= r];

# portfolio constraint
constraints += [sum(x_pos) - sum(x_neg) == 1]
constraints += [sum(x_neg) <= 4]
constraints += [x_pos >= 0, x_neg >=0]

# absolute value constraints
#constraints += [cvx.abs(x) <= B]

prob = cvx.Problem(objective,constraints)
prob.solve()

5.982651221281714

In [5]:
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: ' + str(prob.value));
    print('Variable values: ')
    print(x.value)

Problem status: optimal
Problem value: 5.982651221281714
Variable values: 
[-4.35808969e-01 -8.00049116e-01 -1.13320116e-11  1.19776860e-01
  1.40345128e-11 -5.00572300e-11 -5.09401398e-12  3.43523326e-01
  7.73760057e-12  1.61878843e-11  2.39293265e-01  8.42466875e-11
 -1.02141071e-11  3.40715961e-12  1.92048538e-11  5.78437737e-12
  2.68510102e-12  9.03890576e-12 -9.51204095e-12 -3.05560169e-11
 -5.58506781e-01  5.32346866e-12 -1.32815520e-11  9.02206603e-12
 -1.14840335e-01  1.98542589e-11  2.35316596e-11  2.22629643e-11
  4.26152658e-12  2.91752362e-01  9.87125217e-11  1.72705056e-12
 -1.08664629e-11  4.68560340e-11  1.08047525e+00 -5.55291496e-12
 -6.40196721e-12  4.81364734e-11 -1.02529267e+00 -4.08859067e-12
  2.80730886e-11  2.28332126e-12  1.99296301e-01  4.58321322e-10
 -2.00802651e-12  3.06699181e-11  9.26227579e-02 -2.64385800e-12
  3.41121187e-11  1.06156537e-01  2.10884578e-11  8.50427856e-12
 -1.10419183e-11 -5.28826960e-12 -7.88074563e-12  7.79046227e-12
  3.97497312e-1